In [1]:
%matplotlib inline

import numpy as np
import pylab as pl
import healpy as hp

from scipy import interpolate

direc_shears = "/gpfs/mira-fs0/projects/DarkUniverse_esp/prlarsen/dtfe_maps/or/rt/"

In [2]:
def loadinWeakLensingMaps(step):
    try:
        a00 = hp.read_map(direc_shears+'rt_'+step+'_A1_2.fits',0)
        a01 = hp.read_map(direc_shears+'rt_'+step+'_A1_2.fits',1)
        a10 = hp.read_map(direc_shears+'rt_'+step+'_A2_2.fits',0)
        a11 = hp.read_map(direc_shears+'rt_'+step+'_A2_2.fits',1)
    except:
        print("Can not find SnapShot %s"%step)
        return None

    kappa_m  = 1.0 - (a00 + a11)*0.5 # kappa map
    shear1_m =  (a11-a00)*0.5        # shear1 map
    shear2_m = -(a01+a10)*0.5        # shear2 map

    return kappa_m, shear1_m, shear2_m


def weights_nz_dz(zs_arr):
    '''
        There are <nstep> redshift bins for sources, 
        - the zeroth bin is [0, (zs[0]+zs[1])/2); zbin = zs[0].
        - the last bin is [(zs[-2]+zs[-1])/2), 1.0); zbin = zs[-1]
        - the ith bin is [(zs[i-1]+z[i])/2, (zs[i]+z[i+1])/2); zbin = zs[i]
        By calculate the areas in the bins, 
        we can obtain the weights for the source planes when summing then up.
    '''
    zs_in, ng_in = np.loadtxt("./source_1.txt", usecols=(0, 1), unpack=True) # n(z) from Joe Zuntz
    dzs = zs_in[1:]-zs_in[:-1]
    hng = (ng_in[1:]+ng_in[:-1])*0.5
    area_tot = np.sum(hng*dzs)
    ng_rescale = ng_in/area_tot          # normalization
    f = interpolate.interp1d(zs_in, ng_rescale) # the function created by interpolation
    
    fnz_norm = f(zs_arr)
    
    wt_arr = zs_arr*0.0
    
    for i in range(len(zs_arr)):
        if i==0:
            zs_1 = 0.0
            zs_m = zs_arr[0]
            zs_2= (zs_arr[0]+zs_arr[1])*0.5
            as1 = (fnz_norm[0]+0.0)*(zs_m-zs_1)*0.5
            as2 = (f(zs_2)+fnz_norm[0])*(zs_2-zs_m)*0.5
        elif i==(len(zs_arr)-1):
            zs_1= (zs_arr[-2]+zs_arr[-1])*0.5
            zs_m= zs_arr[-1]
            zs_2= 1.0
            as1 = (fnz_norm[-1]+f(zs_1))*(zs_m-zs_1)*0.5
            as2 = (f(zs_2)+fnz_norm[-1])*(zs_2-zs_m)*0.5
        else:
            zs_1= (zs_arr[i]+zs_arr[i-1])*0.5
            zs_m= zs_arr[i]
            zs_2= (zs_arr[i]+zs_arr[i+1])*0.5
            as1 = (fnz_norm[i]+f(zs_1))*(zs_m-zs_1)*0.5
            as2 = (f(zs_2)+fnz_norm[i])*(zs_2-zs_m)*0.5

        wt_arr[i] = as1+as2

    return wt_arr

In [10]:
def cal_eff_wl_maps(saveToFile=False):
    # Initial the redshifts of source planes
    data_list = {}
    data_list['steps'] = np.array(['487', '475', '464', '453', '442', '432', '421', '411',
                                   '401', '392', '382', '373', '365', '355', '347', '338',
                                   '331', '323', '315', '307', '300', '293', '286', '279', 
                                   '272', '266', '259', '253', '247', '241', '235', '230', 
                                   '224', '219', '213', '208', '203', '198', '194', '189',
                                   '184', '180', '176', '171', '167', '163', '159', '155', 
                                   '151', '148', '144', '141', '137', '134', '131', '127', 
                                   '124', '121'])
    data_list['zs'] = np.array([0.019587576, 0.04249394, 0.06348896, 0.08982432, 0.11601174, 
                                0.14167726, 0.17023492, 0.19876349, 0.22816956, 0.25715542, 
                                0.28774858, 0.31976724, 0.3496554, 0.38333392, 0.4181242, 
                                0.45316005, 0.4872712, 0.5210507, 0.5587207, 0.5984776, 
                                0.63728297, 0.67555356, 0.7154497, 0.7575482, 0.8018985, 
                                0.8445264, 0.8898808, 0.9368595, 0.9829167, 1.030879, 
                                1.0813811, 1.1297112, 1.1807017, 1.2338057, 1.2901752, 
                                1.3487234, 1.4050395, 1.4638758, 1.5192988, 1.5775385, 
                                1.6453991, 1.7094843, 1.7690942, 1.83975, 1.9136083, 
                                1.9828148, 2.0554118, 2.131457, 2.2114959, 2.2847886, 
                                2.3618498, 2.4422584, 2.5272007, 2.6155977, 2.6953585, 
                                2.7935054, 2.895901, 2.9888024])

    nsteps = len(data_list['zs'])
    
    weights = weights_nz_dz(data_list['zs'])

    kappa_map_last, shear1_map_last, shear2_map_last = loadinWeakLensingMaps(data_list['steps'][-1])
    kappa_map = kappa_map_last*weights[-1]
    shear1_map = shear1_map_last*weights[-1]
    shear2_map = shear2_map_last*weights[-1]

    for i in range(nsteps-1):

        res_tmp = loadinWeakLensingMaps(data_list['steps'][i])
        
        if res_tmp==None:
            continue
        else:
            kappa_m_tmp, \
            shear1_m_tmp, \
            shear2_m_tmp = res_tmp

        kappa_map  = kappa_map  + kappa_m_tmp*weights[i]
        shear1_map = shear1_map + shear1_m_tmp*weights[i]
        shear2_map = shear2_map + shear2_m_tmp*weights[i]
        
    #-----------------------------------
    # save maps
    #  
    if saveToFile:
        hp.fitsfunc.write_map('WL_MAPs.fits',m=(kappa_map,shear1_map,shear2_map),overwrite=True)
        
    return kappa_map, shear1_map, shear2_map
        

In [11]:
def vis_kappa_eff(kappa_map):
    nside = 4096
    #-----------------------------------
    # Create masks according to the octant properties
    #

    x,y,z = hp.pix2vec(nside, np.arange(hp.nside2npix(nside)))
    mask_octant = (x>0)&(y>0)&(z<0)
    #-----------------------------------
    # setup the colormap
    #

    cmap = pl.cm.jet
    cmap.set_over(cmap(1.0))
    cmap.set_under('w')
    cmap.set_bad('gray')
    #-----------------------------------
    # make a plot in healpix map
    #

    kappa_map = hp.ma(kappa_map)
    kappa_map.mask = np.logical_not(mask_octant)
    hp.mollview(kappa_map, cmap=cmap)
    
    return 0

In [ ]:
%%time

ka_map, s1_map, s2_map = cal_eff_wl_maps(saveToFile=True)

NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
Can not find SnapShot 487
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = RING in fits file
IN